In [1]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install pandas
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 157.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 132.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 69.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 103.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 192.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 74.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 97.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 123.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 116.9 MB/s 

In [16]:
import transformers
import torch
import pandas as pd
from huggingface_hub import login
import re

In [3]:
# Authenticate with Huggingface token
login()

In [4]:
# Load model pipeline
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Melding når jeg kører apply på datasæt i df-format:
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset

In [5]:
# Load data
data = pd.read_csv('preprocessed_test_data.csv', usecols = ['text', 'label'])
print(len(data))
data.head()

2964


,text,label
0,"DMI: Vil føles som minus 15 grader. Sne, slud,...",Neutral
1,Efter skyderi i boligkvarter: Teenager anholdt...,Negative
2,Efter anholdelser: Drabsmistænkte teenagere sk...,Negative
3,Spansk domstol frikender 36 spillere i matchfi...,Neutral
4,BIF reagerer på boykot: Vil forbedre sikkerhed...,Negative


In [6]:
# Function to generate LLM annotations
def generate_annotation(string):
    messages = [
        {"role": "system", "content": """
        Du er ekspert i sentiment for nyhedsartikler.Givet overskriften og underoverskriften på en artikel, skal du tildele den et label svarende til det sentiment den fremkalder. 
        Du vurderer artiklens overordnede sentiment og fortolker artiklen som en gennemsnitlig dansk nyhedslæser.
        Mulige labels: Positive, Negative, Neutral.
        Giv et kort og præcist svar i dette format: {sentiment: "label"}.
        Label-beskrivelser:
        "Positive": Fremkalder en overordnet positiv sentiment. Stemninger som optimisme, tilfredshed og selvsikkerhed betragtes som positive. 
        "Negative": Fremkalder en overordnet negativ sentiment. Stemninger som vrede, skuffethed og tristhed betragtes som negative.
        "Neutral": Fremkalder hverken en positiv eller negativ sentiment. Enten ingen sentiment eller tvetydig sentiment.        
        """},
        {"role": "user", "content": f"Artikeloverskrift- og underoverskrift:: {string}. Det overordnede sentiment i denne artikel er: "},
    ]

    output = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Return the generated content
    return output[0]["generated_text"][-1]['content']

In [7]:
# Apply the function to each element in the 'text' column
data['LLM_annotation'] = data['text'].apply(generate_annotation)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

In [8]:
data['LLM_annotation'].head()

0    {sentiment: "Negative"}
1    {sentiment: "Negative"}
2     {sentiment: "Neutral"}
3     {sentiment: "Neutral"}
4    {sentiment: "Positive"}
Name: LLM_annotation, dtype: object

In [9]:
# Check element types
types = data['LLM_annotation'].apply(type)
types.value_counts()

LLM_annotation
<class 'str'>    2964
Name: count, dtype: int64

In [17]:
# Post-processing. Clean responses.
def clean_response(text):
    # Use regex to extract the sentiment pattern
    match = re.search(r'\{sentiment: "[^"]+"\}', text)
    if match:
        return match.group(0)
    return text

In [18]:
data['LLM_annotation'] = data['LLM_annotation'].apply(clean_response)

In [22]:
# Function to extract the sentiment label
def extract_sentiment_label(text):
    # Use regex to capture the sentiment label inside the quotes
    match = re.search(r'sentiment: "([^"]+)"', text)
    if match:
        return match.group(1)  # Extract the text inside the quotes
    return None

In [23]:
data['LLM_annotation'] = data['LLM_annotation'].apply(extract_sentiment_label)

In [24]:
data['LLM_annotation']

0       Negative
1       Negative
2        Neutral
3        Neutral
4       Positive
          ...   
2959    Negative
2960    Positive
2961    Positive
2962    Positive
2963    Negative
Name: LLM_annotation, Length: 2964, dtype: object

In [25]:
data.head()

,text,label,LLM_annotation
0,"DMI: Vil føles som minus 15 grader. Sne, slud,...",Neutral,Negative
1,Efter skyderi i boligkvarter: Teenager anholdt...,Negative,Negative
2,Efter anholdelser: Drabsmistænkte teenagere sk...,Negative,Neutral
3,Spansk domstol frikender 36 spillere i matchfi...,Neutral,Neutral
4,BIF reagerer på boykot: Vil forbedre sikkerhed...,Negative,Positive


In [29]:
# Calculate accuracy

# Calculate how many annotations match
correct_annotations = (data['LLM_annotation'] == data['label']).sum()

# Calculate total number of annotations
total_annotations = len(data)

# Calculate accuracy
accuracy = correct_annotations / total_annotations

print("Accuracy: ", round(accuracy, 3))

Accuracy:  0.655


In [30]:
# Save df as .csv-file
data.to_csv('LLM_annotations_Llama3.1Instruct_zeroshot_all_sentiment_test_data.csv')